In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from glob import glob

from tqdm import tqdm_notebook
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import load_model, Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

%matplotlib inline
tf.__version__

'2.5.0'

In [ ]:
data_path='/content/drive/My Drive/data_transferlearning'

In [ ]:

import os

# augmenter le train data et le stocker dans un dossier "preview"
classes = ["Bear","Duck","Eagle","Elephant","Lion","Rabbit","Sheep","Wolf"]
"""
for j in classes:
    monRepertoire = "data_transferlearning/train/"+j
    for file in os.listdir(monRepertoire):
        chemin = os.path.join(monRepertoire, file)
        #print(file)
        img = load_img(chemin) # this is a PIL image
        x = img_to_array(img) # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape) # this is a Numpy array with shape (1, 3, 150, 150)
        #print(x.shape)
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir='data_transferlearning/preview/'+j, save_prefix=j.lower(), save_format='jpeg'):
            i += 1
            if i > 20:
                break
print("end")
"""

In [ ]:
train_dir = os.path.join(data_path, "preview")
test_dir = os.path.join(data_path, "validation")

In [ ]:
IMG_SHAPE = (150, 150, 3)

In [ ]:
# récuperer des models pour le transfert learning "https://keras.io/api/applications/"
base_model = VGG16(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
base_model.trainable = False

In [ ]:
# nombre de sortie du modele
base_model.output

<KerasTensor: shape=(None, 4, 4, 512) dtype=float32 (created by layer 'block5_pool')>

In [ ]:
# utile pour obtenir le nombre de classes
folders = glob(train_dir+'/*')

In [ ]:
folders

['/content/drive/My Drive/data_transferlearning/preview/Bear',
 '/content/drive/My Drive/data_transferlearning/preview/Duck',
 '/content/drive/My Drive/data_transferlearning/preview/Eagle',
 '/content/drive/My Drive/data_transferlearning/preview/Elephant',
 '/content/drive/My Drive/data_transferlearning/preview/Lion',
 '/content/drive/My Drive/data_transferlearning/preview/Rabbit',
 '/content/drive/My Drive/data_transferlearning/preview/Sheep',
 '/content/drive/My Drive/data_transferlearning/preview/Wolf']

In [ ]:
# on peut ajouter d'autres couches(layers) 
x = Flatten()(base_model.output)
# x = Dense(1000, activation='relu')(x)
#x = base_model.output
#le nombre de classes
print("nombre de classes = ", len(folders))
prediction = Dense(len(folders), activation='softmax')(x)

nombre de classes =  8


In [ ]:
model = Model(inputs=base_model.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
model.compile(  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

In [ ]:
# Générez des lots de données d'images de tenseurs avec une augmentation des données en temps réel.
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 classes = ["Bear","Duck","Eagle","Elephant","Lion","Rabbit","Sheep","Wolf"],
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (150, 150),
                                            batch_size = 32,
                                            classes = ["Bear","Duck","Eagle","Elephant","Lion","Rabbit","Sheep","Wolf"],
                                            class_mode = 'categorical')

Found 12544 images belonging to 8 classes.
Found 157 images belonging to 8 classes.


In [ ]:
! pip install tqdm

In [ ]:
model.output

<KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'dense')>

In [ ]:
model.fit_generator(training_set, epochs=50, validation_data=test_set)
model.save('/content/drive/My Drive/data_transferlearning/model5_augmentation_data(50epochs).h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
392/392 [==============================] - 144s 311ms/step - loss: 0.7435 - accuracy: 0.7552 - val_loss: 0.2531 - val_accuracy: 0.8981
Epoch 2/50
392/392 [==============================] - 116s 296ms/step - loss: 0.2092 - accuracy: 0.9363 - val_loss: 0.1544 - val_accuracy: 0.9490
Epoch 3/50
392/392 [==============================] - 117s 299ms/step - loss: 0.1312 - accuracy: 0.9641 - val_loss: 0.1855 - val_accuracy: 0.9236
Epoch 4/50
392/392 [==============================] - 115s 292ms/step - loss: 0.0946 - accuracy: 0.9755 - val_loss: 0.2106 - val_accuracy: 0.8981
Epoch 5/50
392/392 [==============================] - 116s 296ms/step - loss: 0.0769 - accuracy: 0.9794 - val_loss: 0.1573 - val_accuracy: 0.9172
Epoch 6/50
392/392 [==============================] - 115s 293ms/step - loss: 0.0765 - accuracy: 0.9788 - val_loss: 0.1946 - val_accuracy: 0.9236
Epoch 7/50
392/392 [==============================] - 116s 296ms/step - loss: 0.0555 - accuracy: 0.9877 - val_loss: 0.2152 -

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(test_set)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9235668778419495
